### LLMs conversation

In [1]:
# Read all text files in data//persona directory
import os
import glob

# Define the directory containing the text files
directory = 'data//persona'
# Use glob to find all text files in the directory
text_files = glob.glob(os.path.join(directory, '*.txt'))
# Initialize an empty dictionary to store the contents of each file
# the dictionary will map file names to their contents
file_contents = {}
# Iterate over each text file
for file_path in text_files:
    # Extract the file name without the directory path
    file_name = os.path.basename(file_path)
    # Open the file and read its contents
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    # Store the content in the dictionary
    file_contents[file_name] = content

In [2]:
import os
import getpass
from llama_index.llms.openrouter import OpenRouter
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Prompt for key if missing
if not os.environ.get("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass.getpass("OPENROUTER_API_KEY: ")

if not os.environ.get("MODEL_NAME"):
    os.environ["MODEL_NAME"] = getpass.getpass("MODEL_NAME: ")

OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]
MODEL_NAME = os.environ["MODEL_NAME"]

# This functions returns a new instance of OpenRouter LLM with the specified model.
def create_llm(model_name: str = MODEL_NAME) -> OpenRouter:
    return OpenRouter(
        api_key=OPENROUTER_API_KEY,
        model=model_name,
        is_chat_model=True,
        is_function_calling_model=True,
    )

In [3]:
from llama_index.core.workflow import Context

# Create an LLM instance for each item in file_contents
llm_instances = {file_name: create_llm() for file_name in file_contents.keys()}

# Create a dictionary for agent instances
# This will map file names to their corresponding FunctionAgent instances
agent_instances = {}

# Create FunctionAgent instances for each LLM with the corresponding file content as system prompt
from llama_index.core.agent.workflow import FunctionAgent
for file_name, content in file_contents.items():
    llm = llm_instances[file_name]
    agent = FunctionAgent(
        llm=llm,
        system_prompt=f"You are a helpful assistant.Answer at max in two sentences in Persian. Your persona is based on the following text: {content}",
    )
    # Store the agent instance in the dictionary
    agent_instances[file_name] = agent

# Create a disctionary for context instances
context_instances = {}
# Create Context instances for each agent
for file_name, agent in agent_instances.items():
    context = Context(agent)
    # Store the context instance in the dictionary
    context_instances[file_name] = context


In [4]:
# Loop 10 times to demonstrate conversation with each agent
for i in range(3):
    for file_name, ctx in context_instances.items():
        # Get the agent from the context
        agent = agent_instances[file_name]
        # Ask a question to the agent
        question = f"Question {i+1} for {file_name}: What is your opinion on Palestine?"
        print(f"{file_name} received question: {question}")
        response = await agent.run(question, ctx=ctx)
        print(f"Response from {file_name}: {response}\n")

Donald Trump.txt received question: Question 1 for Donald Trump.txt: What is your opinion on Palestine?
Response from Donald Trump.txt: فلسطین؟ یک وضعیت بسیار پیچیده است، یک فاجعه. ما در دوران من کارهای زیادی برای صلح انجام دادیم، توافقات ابراهیم را ببینید، بی‌نظیر بودند. هیچ‌کس فکر نمی‌کرد ممکن باشد، اما دونالد جی. ترامپ این کار را کرد.

Joe Biden.txt received question: Question 1 for Joe Biden.txt: What is your opinion on Palestine?
Response from Joe Biden.txt: ببینید، قضیه اینه که من عمیقاً به راه‌حل دو دولتی اعتقاد دارم. این تنها راهیه که می‌تونه صلح و امنیت رو هم برای اسرائیلی‌ها و هم برای فلسطینی‌ها به ارمغان بیاره.

Donald Trump.txt received question: Question 2 for Donald Trump.txt: What is your opinion on Palestine?
Response from Donald Trump.txt: فلسطین؟ یک وضعیت بسیار پیچیده است، یک فاجعه. ما در دوران من کارهای زیادی برای صلح انجام دادیم، توافقات ابراهیم را ببینید، بی‌نظیر بودند. هیچ‌کس فکر نمی‌کرد ممکن باشد، اما دونالد جی. ترامپ این کار را کرد.

Joe Biden.txt received quest

### Using AutoTokenizer to tokenize text

In [5]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("qwen/qwen2.5-vl-72b-instruct")

# Example usage
text = "Hello, how are you?"
encoded_input = tokenizer(text, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[9707,   11, 1246,  525,  498,   30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
